# setup

In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

from efficientnet_pytorch import EfficientNet
from PIL import Image
from torchvision import transforms

from transformers import AutoModel, AutoTokenizer
import torch
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch, gc

from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances

HOME = '/data/git/shopee-product-matching'
pdata = f'{HOME}/data/shopee-product-matching'
!ls $pdata

BS = 256
NWKRS = 8
DEVICE = 'cuda'
PIN_MEMORY = True
MAXLEN = 128

sample_submission.csv  test.csv  test_images  train.csv  train_images


# data

In [2]:
fnm = 'train'
df = pd.read_csv(f'{pdata}/{fnm}.csv')
assert len(df) == df.posting_id.nunique()
p_imgs = f"{pdata}/train_images"
df.head(2)

,posting_id,image,image_phash,title,label_group
0,train_129225211,0000a68812bc7e98c42888dfb1c07da0.jpg,94974f937d4c2433,Paper Bag Victoria Secret,249114794
1,train_3386243561,00039780dfc94d01db8676fe789ecd05.jpg,af3f9460c2838f0f,"Double Tape 3M VHB 12 mm x 4,5 m ORIGINAL / DO...",2937985045


# hash

In [3]:
# https://leetcode.com/problems/hamming-distance/discuss/1105813/Python3-with-bit-manipulation-faster-than-99.96.
def hammingDistance(x: int, y: int) -> int:
    n=x^y     #taking XOR of inputs to capture the difference(at bit level) between them.
    counter=0
    while(n):           #to count the set bits
        counter+=1
        n&=(n-1)
    return counter

hfeats = [int(x, 16) for x in df.image_phash]
# list(df.image_phash.apply(lambda x: int(x, 16)).values)

In [4]:
hammingDistance(10707114133977572403, 12627975023471333135)

42

In [5]:
n = len(hfeats)
hdists = np.zeros((n, n))
for i in tqdm(range(n)):
    for j in range(i, n):
        x1 = hfeats[i]
        x2 = hfeats[j]
        hdists[i,j]=hdists[j,i]=hammingDistance(x1, x2)

  7%|▋         | 2332/34250 [05:19<1:12:59,  7.29it/s]


KeyboardInterrupt: 

In [ ]:
hsims = 1/(1+hdists)

In [ ]:
hsims

# demo

In [ ]:
i = np.random.choice(range(len(df)))
print(i)
df.iloc[i].to_dict()

In [ ]:
grp = df.iloc[i].label_group

df[df.label_group==grp][['title', 'image_phash', 'label_group']]

In [ ]:
ntop = 10

In [ ]:
idx_sim = np.argsort(-hsims[i])[:ntop]
df.iloc[idx_sim][['title', 'image_phash', 'label_group']]

# save 

In [ ]:
pout = f"{HOME}/data/dev0008"
!mkdir -p $pout

In [ ]:
np.save(f"{pout}/hdists.npy", hdists)
np.save(f"{pout}/hsims.npy", hsims)

        title	image_phash	label_group
    34221	Sticker Star Wall Glow In The Dark | Stiker Bi...	afde42252f25c992	3251156563
    10216	Hiasan Dinding 100 PC Bintang Star Cantik LED ...	afde42250f25c9d2	3251156563

In [ ]:
def hamming(x1, x2): 
    x1,x2 = (bin(int(x, 16)) for x in (x1, x2))
    return sum(o1!=o2 for o1,o2 in zip(x1,x2))

In [ ]:
hamming("afde42252f25c992", "afde42250f25c9d2")

In [ ]:
%%timeit
hamming("afde42252f25c992", "afde42250f25c9d2")

In [ ]:
# https://www.geeksforgeeks.org/hamming-distance-between-two-integers/
def hammingDistance(n1, n2) : 
    x = n1 ^ n2  
    setBits = 0
    while (x > 0) : 
        setBits += x & 1
        x >>= 1
    return setBits  

In [ ]:
hammingDistance(0xafde42252f25c992, 0xafde42250f25c9d2)

In [ ]:
%%timeit
hammingDistance(0xafde42252f25c992, 0xafde42250f25c9d2)

In [ ]:
# https://leetcode.com/problems/hamming-distance/discuss/1105813/Python3-with-bit-manipulation-faster-than-99.96.
def hammingDistance(x: int, y: int) -> int:
    n=x^y     #taking XOR of inputs to capture the difference(at bit level) between them.
    counter=0
    while(n):           #to count the set bits
        counter+=1
        n&=(n-1)
    return counter

In [ ]:
hammingDistance(0xafde42252f25c992, 0xafde42250f25c9d2)

In [ ]:
%%timeit
hammingDistance(0xafde42252f25c992, 0xafde42250f25c9d2)